In [ ]:
import mne
from mne import find_events
import numpy as np
import os
from mne.preprocessing import ICA, create_ecg_epochs, read_ica

In [ ]:
s_id = '13'
tmin, tmax = -0.1, 0.6

print('Subject id: ', s_id)
print('tmin = ', tmin)
print('tmax = ', tmax)

meg_raw_MainFolder = "..\Data\MEG_Data\RawData\\"
ica_MainFolder = "..\icaModels\\"

blocks = [0,1, 2]

triggers_cry_real = [9, 10, 11] # By block: sound 1 + nu_consistent 
triggers_phone_real = [11, 12, 13] # By block: sound 3 + nu_consistent 

triggers_cry_omi = [14, 15, 16] # By block: sound 1 + nu_consistent + omission code (5)
triggers_phone_omi = [16, 17, 18] # By block: sound 3 + nu_consistent + omission code (5)
pred_all = [8, 9, 10]

raw_all = []
print('x---------------x')


#### Load Raw data   

In [ ]:

for bl in blocks:
    print('Block: ', bl)
    
    if int(s_id) > 22:
        if s_id == '38' or s_id == '39' or s_id == '40' or s_id == '41':
            filename = 'P' + s_id + '\\block_' +  str(bl) + '_tsss_mc_trans.fif'
            fraw = meg_raw_MainFolder + filename
            
        else:
            filename = 'S' + s_id + '\\Block_' + str(bl) + '_tsss_mc_trans.fif' # s_id +'_' + str(bl) OR 'block_' + str(bl) + '_tsss_mc_trans.fif' 
            fraw = meg_raw_MainFolder + filename
           
                
    else:
        filename = 'S' + s_id + '\\' + s_id + '_' + str(bl) + '_tsss_mc_trans.fif'
        fraw = meg_raw_MainFolder + filename
    
    
    if os.path.isfile(meg_raw_MainFolder) == True:
         print(fraw)
    else:
        print('No file named as ', fraw)
    
    
    if os.path.isfile(fraw):
        # read and filter raw data:
        raw = mne.io.read_raw_fif(fraw , preload = True)
        raw.info['ch_names'] = [chn.strip('.') for chn in raw.info['ch_names']]
        
        print("Data imported!")
        
        raw.filter(0.1, 40., fir_design='firwin')
        print("Data filtered!")
        raw_all.append(raw)

        del raw        

In [ ]:
nchan_list = []
for ind, r in enumerate(raw_all):
    print('Block id = %d -- Number of channels = %d' %(ind, r.info['nchan']))
    nchan_list.append(r.info['nchan'])

In [ ]:
nchan_list

!!! If number of channels are not equal in all block, remove the extra ones to be able to concatenate the raw data

In [ ]:
nchan_max = nchan_list.index(max(nchan_list))
nchan_max

In [ ]:
channels_toRemove = []
for i in range(nchan_list[nchan_max]):
    if raw_all[nchan_max].info['ch_names'][i]  not in  raw_all[0].info['ch_names']:
        channels_toRemove.append(raw_all[nchan_max].info['ch_names'][i])
channels_toRemove

In [ ]:
#drop the channels in channels_toRemove list
if len(channels_toRemove) > 0:
    raw_all[nchan_max].drop_channels(channels_toRemove)
    raw_all[nchan_max].info['nchan']
else:
    print('Nothing to remove!')

In [ ]:
# Print the shape of data in each block
for r in raw_all:
    print('-------')
    print(r.get_data().shape)



In [ ]:
# Concatenate raw data from each block
raw = mne.io.concatenate_raws(raw_all)
print(raw.get_data().shape)
del raw_all

Visualize EOG

In [ ]:
# Visualize eog(eye movement) noise 
import matplotlib
%matplotlib inline

eog_evoked = mne.preprocessing.create_eog_epochs(raw).average()
eog_evoked.plot_joint()

Visualize ECG

In [ ]:
#Visualize ECG noise (cardiac noise)
ecg_evoked = mne.preprocessing.create_ecg_epochs(raw).average()
ecg_evoked.plot_joint()

ICA is sensitive to low-frequency drifts and therefore requires the data to be high-pass filtered prior to fitting. Typically, a cutoff frequency of 1 Hz is recommended. However, because filtering is a linear operation, the ICA solution found from the filtered signal can be applied to the unfiltered signal [https://mne.tools/stable/auto_tutorials/preprocessing/plot_40_artifact_correction_ica.html#what-is-ica]


In [ ]:
ica_file = ica_MainFolder + 'S' + s_id + '\ica.fif'
print(ica_file)       

In [ ]:
# Building ica model
if os.path.isfile(ica_file) is True:
    ica = read_ica(ica_file)
else:
    filt_raw = raw.copy()
    filt_raw.load_data().filter(l_freq=1., h_freq=None)
    print(filt_raw.get_data().shape)
    
    #Generate ica model
    ica = ICA(n_components=0.95, method='fastica', random_state=97)
    ica.fit(filt_raw)
    #Save model
    ica.save(ica_file)
    
    del filt_raw

#### Copy data

In [ ]:
orig_raw = raw.copy()

In [ ]:
raw.load_data()

#### Visualize sources

In [ ]:
import mne.viz
import matplotlib as plt
%matplotlib tk
ica.plot_sources(raw, stop=20)